# Sanity check
## 1. Cel sprawozdania
Celem tego pliku jest dokumentacja kodu oraz przebadanie poprawności naszej symulacji poprzez porównanie z teoretycznym, spodziewanym wynikiem. 
Na końcu znajdują się wyniki analizy statystycznej po wielokrotnym uruchomieniu symulacji dla prostego przykładu.
## 2. Dokumentacja kodu

In [2]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import pandas as pd
import simpy
import seaborn
import matplotlib.pyplot as plt

Każdy obiekt w naszej symulacji jest klasą, która przechowuje informacje o sobie.

Dla pacjenta są to:
- `id`: identyfikator Pacjenta
- `arrival_time`: Godzina, o której przyszedł
- `service_start_time`: Godzina, o której wszedł do gabinetu (rozpoczęła się obsługa)
- `service_end_time`: Godzina, o której wyszedł z gabinetu (zakończenie obsługi)

Każda zmienna **`time`** jest przechowywana jako liczba minut od rozpoczęcia symulacji

In [3]:
class Pacjent:
    def __init__(self, id):
        self.day = 1
        self.id = id
        self.room = None
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None

Dla gabinetu są to:
- `id`: identyfikator gabinetu
- `env`: środowisko symulacji
- `patients_served`: liczba obsłużonych pacjentów

In [4]:
class Gabinet:
    def __init__(self, id):
        self.id = id
        self.patients_served = 0

## Parametry kliniki:
- **`number_of_ rooms`**: liczba gabinetów
- **`service_time`**: Jak długo obsługiwany jest pacjent [minuty]
- **`lambda_per_hour`**: Ile pacjentów przychodzi co godzinę (dzieli się przez 60 bo zamieniamy na minuty)
- **`sim_time`**: Długość symulacji

## Pomocnicze zmienne:
- **`queue_array`**: Kolejka (podgląd czekających pacjentów)
- **`seed`**: seed generatora liczb pseudolosowych
- **`list_rooms`**: lista gabinetów o długości **`number_of_rooms`**
- **`rooms`**: struktura Store z biblioteki simpy o wielkości **`number_of_rooms`**
- **`processed_patients`**: lista obsłużonych już pacjentów

In [5]:
class Clinic:
    def __init__(self, number_of_rooms, service_time, queue_array=[], lambda_per_hour=6/60, seed=None, sim_time=120):
        self.curr_patient_id = 1
        self.env = simpy.Environment()
        self.service_time = service_time
        self.lambda_per_hour = lambda_per_hour
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1) for i in range(number_of_rooms)]
        self.rooms = simpy.Store(self.env, capacity=number_of_rooms)
        self.sim_time = sim_time
        self.queue_array = queue_array
        for room in self.list_rooms:
            self.rooms.put(room)
        self.processed_patients = []


### Metoda `czas()`:
Pomocnicza funkcja do ładnego wypisywania czasu.

Parametr `time_in_minutes`: czas w minutach

Praca zaczyna się od godziny 9:00 dlatego hours = 9+[...]

In [6]:

def czas(self, time_in_minutes):
    hours = 9+time_in_minutes//60
    minutes = time_in_minutes%60
    if len(str(int(minutes)))==1:
        return f"{int(hours)}:0{int(minutes)}"
    return f"{int(hours)}:{int(minutes)}"
Clinic.czas = czas

### Metoda `time_between_new_patients()`:
Ważna metoda zwracająca przerwę w minutach, do pojawienia się następnego pacjenta, podawana jest jako parametr metody `env.timeout()` po wygenerowaniu pacjenta.

Wykorzystuje rozkład wykładniczy o intensywności $\frac{1}{\lambda}$

In [7]:
def time_between_new_patients(self):
    if self.seed:
        np.random.seed(self.seed)
    return np.random.exponential(1/self.lambda_per_hour)
Clinic.time_between_new_patients = time_between_new_patients

### Metoda `generate_patients()`:

Metoda wygeneruje pacjenta o jakimś id i rozpocznie dla niego proces `serve_patient()`, aż do momentu {service_time} przed zamknięciem (Uznaliśmy, że pacjenci nie przychodzą np. 15 minut przed zamknięciem). Dzięki niej upłynie czas do przyjścia nastepnego pacjenta

In [8]:
def generate_patients(self):
    while True:
        if self.env.now >= self.sim_time-self.service_time: # Pacjenci nie przychodzą {service_time} przed zamknięciem
            break
        patient = Pacjent(id=self.curr_patient_id)
        patient.arrival_time = self.env.now
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} przybył do kliniki")
        self.env.process(self.serve_patient(patient))
        self.curr_patient_id += 1
        yield self.env.timeout(self.time_between_new_patients())
Clinic.generate_patients = generate_patients

### Metoda `serve_patient(patient)`

Odpalana dla jakiegoś pacjenta:
- Sprawdza czy jakiś pokój jest wolny i dodaje go do podglądu kolejki (queue_array) (Taka prawdziwa kolejka jest wbudowana w SimPy i uruchamiana jest przez `rooms.get()`)
- Jeśli mija godzina *{service_time}* od końca pracy (sim_time) wtedy pacjenci są usuwani z kolejki.
- Obsługuje pacjentów i puszcza czas dalej.

In [9]:
def serve_patient(self, patient): 
    room = yield self.rooms.get()
    self.queue_array.append(patient.id)
    if self.env.now >= self.sim_time-self.service_time: # Nie zdążyło obsłużyć pacjenta przed zamknięciem
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} - nie zdążył zostać obsłużony przed zamknięciem kliniki")
        yield self.rooms.put(room)
        return
    patient.service_start_time = self.env.now
    print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
    self.queue_array.remove(patient.id)
    yield self.env.timeout(self.service_time)

    patient.service_end_time = self.env.now
    room.patients_served += 1
    print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wychodzi z gabinetu {room.id}, czekal od {self.czas(np.trunc(patient.arrival_time))} do {self.czas(np.trunc(patient.service_start_time))}")
    patient.room = room.id
    self.processed_patients.append(patient)
    yield self.rooms.put(room)
Clinic.serve_patient = serve_patient

### Metoda `run()` 
Uruchamia symulacje i rozpoczyna proces generowania pacjentów.

In [10]:
def run(self, day=1):
    self.env.process(self.generate_patients())
    self.env.run(until=self.sim_time)
    df = pd.DataFrame([
    {
        "day": day,
        "id": p.id,
        "room": p.room,
        "arrival_time": p.arrival_time,
        "service_start_time": p.service_start_time,
        "service_end_time": p.service_end_time,
        "waiting_time": p.service_start_time - p.arrival_time
    }
    for p in self.processed_patients
    ])
    df = df.set_index("id")
    return df
Clinic.run = run

In [11]:
def run_multiple_times(self, n):
    for i in range(1, n+1):
        print(f"-----------{i}-----------")
        self.env = simpy.Environment()
        single_df = self.run(i)
        multiple_df = pd.concat([multiple_df, single_df], ignore_index=True) if i > 1 else single_df
    return multiple_df
Clinic.run_multiple_times = run_multiple_times


## 3. Analiza statystyczna (statystyki opisowe)

In [12]:
def stats(self):
    def patient_bar_plot():
        patients_served_ls = []
        id_ls = []
        for room in self.list_rooms:
            patients_served_ls.append(room.patients_served)
            id_ls.append(str(room.id))
        fig = seaborn.barplot(x=id_ls, y=patients_served_ls)
        fig.set_xlabel('Gabinet')
        fig.set_ylabel('Ilosc Pacjentow')
        plt.savefig("patients_served_noshow.png")
        plt.show()

    patient_bar_plot()
Clinic.stats = stats

In [13]:
clinic = Clinic(number_of_rooms=2, service_time=15)

In [14]:
clinic.run()

Czas 9:00: Pacjent 1 przybył do kliniki
Czas 9:00: Pacjent 1 wchodzi do gabinetu 1 
Czas 9:12: Pacjent 2 przybył do kliniki
Czas 9:12: Pacjent 2 wchodzi do gabinetu 2 
Czas 9:14: Pacjent 3 przybył do kliniki
Czas 9:15: Pacjent 1 wychodzi z gabinetu 1, czekal od 9:00 do 9:00
Czas 9:15: Pacjent 3 wchodzi do gabinetu 1 
Czas 9:24: Pacjent 4 przybył do kliniki
Czas 9:27: Pacjent 2 wychodzi z gabinetu 2, czekal od 9:12 do 9:12
Czas 9:27: Pacjent 4 wchodzi do gabinetu 2 
Czas 9:30: Pacjent 3 wychodzi z gabinetu 1, czekal od 9:14 do 9:15
Czas 9:31: Pacjent 5 przybył do kliniki
Czas 9:31: Pacjent 5 wchodzi do gabinetu 1 
Czas 9:39: Pacjent 6 przybył do kliniki
Czas 9:42: Pacjent 4 wychodzi z gabinetu 2, czekal od 9:24 do 9:27
Czas 9:42: Pacjent 6 wchodzi do gabinetu 2 
Czas 9:43: Pacjent 7 przybył do kliniki
Czas 9:45: Pacjent 8 przybył do kliniki
Czas 9:46: Pacjent 5 wychodzi z gabinetu 1, czekal od 9:31 do 9:31
Czas 9:46: Pacjent 7 wchodzi do gabinetu 1 
Czas 9:51: Pacjent 9 przybył do klini

,day,room,arrival_time,service_start_time,service_end_time,waiting_time
id,,,,,,
1,1,1,0.000000,0.000000,15.000000,0.000000
2,1,2,12.515027,12.515027,27.515027,0.000000
3,1,1,14.517666,15.000000,30.000000,0.482334
4,1,2,24.951779,27.515027,42.515027,2.563248
5,1,1,31.077032,31.077032,46.077032,0.000000
6,1,2,39.848485,42.515027,57.515027,2.666543
7,1,1,43.351555,46.077032,61.077032,2.725477
8,1,2,45.350044,57.515027,72.515027,12.164984
9,1,1,51.185157,61.077032,76.077032,9.891875


In [15]:
clinic.stats()

In [16]:
multiple_df = clinic.run_multiple_times(3)

-----------1-----------
Czas 9:00: Pacjent 12 przybył do kliniki
Czas 9:08: Pacjent 13 przybył do kliniki
Czas 9:12: Pacjent 14 przybył do kliniki
Czas 9:22: Pacjent 15 przybył do kliniki
Czas 9:30: Pacjent 16 przybył do kliniki
Czas 9:32: Pacjent 17 przybył do kliniki
Czas 9:32: Pacjent 18 przybył do kliniki
Czas 9:48: Pacjent 19 przybył do kliniki
Czas 9:51: Pacjent 20 przybył do kliniki
Czas 9:54: Pacjent 21 przybył do kliniki
Czas 10:04: Pacjent 22 przybył do kliniki
Czas 10:07: Pacjent 23 przybył do kliniki
Czas 10:25: Pacjent 24 przybył do kliniki
-----------2-----------
Czas 9:00: Pacjent 25 przybył do kliniki
Czas 9:02: Pacjent 26 przybył do kliniki
Czas 9:14: Pacjent 27 przybył do kliniki
Czas 9:23: Pacjent 28 przybył do kliniki
Czas 9:27: Pacjent 29 przybył do kliniki
Czas 10:06: Pacjent 30 przybył do kliniki
Czas 10:11: Pacjent 31 przybył do kliniki
Czas 10:33: Pacjent 32 przybył do kliniki
Czas 10:37: Pacjent 33 przybył do kliniki
-----------3-----------
Czas 9:00: Pacjent 

In [17]:
multiple_df.describe()

,day,room,arrival_time,service_start_time,service_end_time,waiting_time
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,2.000000,1.454545,36.828324,40.449659,55.449659,3.621335
std,0.829156,0.505650,22.571108,24.869286,24.869286,4.438778
min,1.000000,1.000000,0.000000,0.000000,15.000000,0.000000
25%,1.000000,1.000000,14.517666,15.000000,30.000000,0.000000
50%,2.000000,1.000000,39.848485,42.515027,57.515027,2.563248
75%,3.000000,2.000000,51.185157,61.077032,76.077032,9.340228
max,3.000000,2.000000,79.140020,79.140020,94.140020,12.164984
